# Using EcoFOCIpy to process raw field data

*SH2202L2 (Year of the Salmon - Shimada)*

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2022/CTDcasts/sh2202l2/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [3]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'SH2202L2' #no hyphens
cruise_meta_file = sample_data_dir+'logs/SH2202L2.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

/Users/bell/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/bell/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/bell/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/bell/miniconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index c

In [4]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd018.btl'].describe()

/var/folders/vv/blq095kj0xj9nz1v0ffdwfch0000gp/T/ipykernel_55183/944620869.py:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  cruise_data['ctd018.btl'].describe()


,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,scan,datetime
count,8.0000,8.000,8.000,8.0000,8.0000,8.000,8.0000,8.0000,8.000,8.000,8.0000,8.0000,8.0000,8.000,8.000,8.0,8
unique,8.0000,8.000,8.000,6.0000,8.0000,6.000,7.0000,8.0000,8.000,8.000,8.0000,8.0000,8.0000,8.000,8.000,8.0,8
top,2.1458,29.531,93.343,32.5926,33.6405,25.916,25.9205,2.0857,28.704,90.728,-0.0428,4.2572,4.2554,100.803,-0.002,6903.0,2022-02-21 17:00:31
freq,1.0000,1.000,1.000,3.0000,1.0000,2.000,2.0000,1.0000,1.000,1.000,1.0000,1.0000,1.0000,1.000,1.000,1.0,1
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-21 17:00:31
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-02-21 17:06:11


## Create BTL report file

In [5]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd018':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [6]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'SH2202L2',
 'CruiseID_Historic': 'SH2202',
 'CruiseID_Alternates': '',
 'Project_Leg': 'L2',
 'Vessel': 'Bell M Shimada',
 'ShipID': 'SH',
 'StartDate': datetime.date(2022, 3, 1),
 'EndDate': datetime.date(2022, 3, 1),
 'Project': 'Year of the Salmon (YOS)',
 'ChiefScientist': '',
 'StartPort': '',
 'EndPort': '',
 'CruiseLocation': 'Gulf of Alaska',
 'Description': '',
 'CruiseYear': 2022,
 'ctdlogs_pdf_name': 'SH2202L2_CastLogs.pdf'}

In [7]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD018']

{'id': 53793,
 'Vessel': 'Bell M Shimada',
 'CruiseID': 'SH2202',
 'Project_Leg': 'L2',
 'UniqueCruiseID': 'SH2202L2',
 'Project': 'International Year of the Salmon',
 'StationNo_altname': '12',
 'ConsecutiveCastNo': 'CTD018',
 'LatitudeDeg': 58,
 'LatitudeMin': 0.23,
 'LongitudeDeg': 146,
 'LongitudeMin': 59.37,
 'GMTDay': 21,
 'GMTMonth': 'Feb',
 'GMTYear': 2022,
 'GMTTime': 60840,
 'DryBulb': -99.9,
 'RelativeHumidity': -999,
 'WetBulb': -99.9,
 'Pressure': -9999,
 'SeaState': '',
 'Visibility': '',
 'WindDir': -999,
 'WindSpd': -99.9,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 4864,
 'StationNameID': '',
 'MaxDepth': 100,
 'InstrumentSerialNos': '',
 'Notes': 'eDNA/HPLC/FA/SI,POM',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '',
 'OxygenBtlNumbers': '',
 'SalinityBtlNiskinNo': '',
 'SalinityBtlNumbers': '',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'W

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [8]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [9]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd018.btl'] = cruise_data['ctd018.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd018.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],turbidity,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,
3.0,7.2046,96.715,313.628,32.5993,32.6047,25.9223,25.9267,7.0617,94.796,307.407,0.5235,3.5229,3.5219,25.659,0.0236,12331.0,2022-02-21 17:04:17,ctd018


In [10]:
cruise_data['ctd018.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'turbidity',
       'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [11]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [12]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd018.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [13]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
Data variables: (12/18)
    oxy_concM_ch1       (bottle) object 2.1458 7.1727 7.2046 ... 7.2115 7.2129
    oxy_percentsat_ch1  (bottle) object 29.531 96.207 96.715 ... 96.826 96.847
    oxy_conc_ch1        (bottle) object 93.343 312.237 ... 313.928 313.991
    salinity_ch1        (bottle) object 33.6351 32.6013 ... 32.5926 32.5926
    salinity_ch2        (bottle) object 33.6405 32.6072 ... 32.5983 32.5979
    sigma_t_ch1         (bottle) object 26.674 25.9271 ... 25.9161 25.916
    ...                  ...
    temperature_ch2     (bottle) object 4.2554 3.487 3.5219 ... 3.5323 3.5328
    Pressure [dbar]     (bottle) object 100.803 50.962 25.659 ... 5.57 5.159
    turbidity           (bottle) object -0.002 0.0296 0.0236 ... 0.0271 0.0285
    scan                (bottle) object 6903.0 10333.0 ... 14916.0 15066.0
    datetime            (bottle) datetime64[ns] 2022-02-21T17:00:31 ... 2022-...
    cast                (bottle) object 'ctd018' 'ctd018' ... 'ctd018' 'ctd018'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [14]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [15]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd018.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [16]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) object 2.1458 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) object 29.531 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) object 93.343 ......
    salinity_ch1        (latitude, longitude, time, bottle) object 33.6351 .....
    salinity_ch2        (latitude, longitude, time, bottle) object 33.6405 .....
    sigma_t_ch1         (latitude, longitude, time, bottle) object 26.674 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) object 4.2554 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) object 100.803 .....
    turbidity           (latitude, longitude, time, bottle) object -0.002 ......
    scan                (latitude, longitude, time, bottle) object 6903.0 ......
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd018' ....

In [17]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD018')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 8, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/18)
    oxy_concM_ch1       (latitude, longitude, time, bottle) object 2.1458 ......
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) object 29.531 ......
    oxy_conc_ch1        (latitude, longitude, time, bottle) object 93.343 ......
    salinity_ch1        (latitude, longitude, time, bottle) object 33.6351 .....
    salinity_ch2        (latitude, longitude, time, bottle) object 33.6405 .....
    sigma_t_ch1         (latitude, longitude, time, bottle) object 26.674 ......
    ...                  ...
    temperature_ch2     (latitude, longitude, time, bottle) object 4.2554 ......
    Pressure [dbar]     (latitude, longitude, time, bottle) object 100.803 .....
    turbidity           (latitude, longitude, time, bottle) object -0.002 ......
    scan                (latitude, longitude, time, bottle) object 6903.0 ......
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd018' ....
Attributes: (12/44)
    CruiseID:                      SH2202L2
    CruiseID_Historic:             SH2202
    CruiseID_Alternates:           
    Project_Leg:                   L2
    Vessel:                        Bell M Shimada
    ShipID:                        SH
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2022-07-11T15:18:48Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [18]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history